# Assignment 2
## Group Members : Sree Divya Keerthi Paravasthu Siddanthi, Tanvi Rasam

In [29]:
from sklearn import linear_model
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import LassoCV
from sklearn.metrics import mean_squared_error, r2_score
import warnings
warnings.filterwarnings("ignore") # to not display any warnings generated 

train = pd.read_csv("dodgers_training.csv")
test = pd.read_csv("dodgers_testing.csv")

In [30]:
train.head()

,Unnamed: 0,month,day,attend,day_of_week,opponent,temp,skies,day_night,cap,shirt,fireworks,bobblehead
0,58,AUG,20,36878,Monday,Giants,80,Clear,Night,NO,NO,NO,NO
1,62,AUG,25,40284,Saturday,Marlins,70,Clear,Night,NO,NO,NO,NO
2,56,AUG,7,55024,Tuesday,Rockies,80,Clear,Night,NO,NO,NO,YES
3,6,APR,23,26376,Monday,Braves,60,Cloudy,Night,NO,NO,NO,NO
4,57,AUG,8,37084,Wednesday,Rockies,84,Clear,Night,NO,NO,NO,NO


In [31]:
test.head()

,Unnamed: 0,month,day,attend,day_of_week,opponent,temp,skies,day_night,cap,shirt,fireworks,bobblehead
0,60,AUG,22,40173,Wednesday,Giants,75,Clear,Night,NO,NO,NO,NO
1,34,JUN,16,45210,Saturday,White Sox,68,Clear,Night,NO,NO,NO,NO
2,55,AUG,6,32659,Monday,Rockies,79,Clear,Night,NO,NO,NO,NO
3,31,JUN,12,55279,Tuesday,Angels,66,Cloudy,Night,NO,NO,NO,YES
4,39,JUL,1,55359,Sunday,Mets,75,Clear,Night,NO,NO,NO,YES


### 1) Encoding categorical Attributes

###       a) To select columns which are needed and splitting dependent(y) and independent variables(x)

In [32]:
x_train = train[['temp','day_of_week','month','skies','bobblehead']]
y_train=train[['attend']]
x_test = test[['temp','day_of_week','month','skies','bobblehead']]
y_test=test[['attend']]

### b) One hot encoding

In [33]:
x_train =pd.get_dummies(x_train)
x_test = pd.get_dummies(x_test)
print(x_train.head())

   temp  day_of_week_Friday  day_of_week_Monday  day_of_week_Saturday  \
0    80                   0                   1                     0   
1    70                   0                   0                     1   
2    80                   0                   0                     0   
3    60                   0                   1                     0   
4    84                   0                   0                     0   

   day_of_week_Sunday  day_of_week_Thursday  day_of_week_Tuesday  \
0                   0                     0                    0   
1                   0                     0                    0   
2                   0                     0                    1   
3                   0                     0                    0   
4                   0                     0                    0   

   day_of_week_Wednesday  month_APR  month_AUG  month_JUL  month_JUN  \
0                      0          0          1          0          0   
1       

## Q1- List the means and standard deviations of the encoded attributes before scaling

In [34]:
print('Means:', x_train.mean())
print('Standard Deviations:', x_train.std())

Means: temp                     73.642857
day_of_week_Friday        0.178571
day_of_week_Monday        0.142857
day_of_week_Saturday      0.160714
day_of_week_Sunday        0.196429
day_of_week_Thursday      0.089286
day_of_week_Tuesday       0.089286
day_of_week_Wednesday     0.142857
month_APR                 0.160714
month_AUG                 0.214286
month_JUL                 0.089286
month_JUN                 0.089286
month_MAY                 0.232143
month_OCT                 0.017857
month_SEP                 0.196429
skies_Clear               0.803571
skies_Cloudy              0.196429
bobblehead_NO             0.910714
bobblehead_YES            0.089286
dtype: float64
Standard Deviations: temp                     8.949207
day_of_week_Friday       0.386459
day_of_week_Monday       0.353094
day_of_week_Saturday     0.370591
day_of_week_Sunday       0.400892
day_of_week_Thursday     0.287736
day_of_week_Tuesday      0.287736
day_of_week_Wednesday    0.353094
month_APR           

### 2) To ensure same attributes in training and testing dataset, we have added thursday in testing dataset

In [35]:
# To make the number of attributes in testing and training data equal
x_test.insert(5, 'day_of_week_Thursday', 0)
x_train.info()
x_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56 entries, 0 to 55
Data columns (total 19 columns):
temp                     56 non-null int64
day_of_week_Friday       56 non-null uint8
day_of_week_Monday       56 non-null uint8
day_of_week_Saturday     56 non-null uint8
day_of_week_Sunday       56 non-null uint8
day_of_week_Thursday     56 non-null uint8
day_of_week_Tuesday      56 non-null uint8
day_of_week_Wednesday    56 non-null uint8
month_APR                56 non-null uint8
month_AUG                56 non-null uint8
month_JUL                56 non-null uint8
month_JUN                56 non-null uint8
month_MAY                56 non-null uint8
month_OCT                56 non-null uint8
month_SEP                56 non-null uint8
skies_Clear              56 non-null uint8
skies_Cloudy             56 non-null uint8
bobblehead_NO            56 non-null uint8
bobblehead_YES           56 non-null uint8
dtypes: int64(1), uint8(18)
memory usage: 1.5 KB
<class 'pandas.core.frame.DataF

### 3) Standardization

In [36]:
# with train data
scaler = preprocessing.StandardScaler(copy=True, with_mean=True, with_std=True).fit(x_train)
scaled_x_train = scaler.transform(x_train)
scaled_x_train = pd.DataFrame(scaled_x_train, columns=x_train.columns)


In [37]:
scaled_x_train.head()

,temp,day_of_week_Friday,day_of_week_Monday,day_of_week_Saturday,day_of_week_Sunday,day_of_week_Thursday,day_of_week_Tuesday,day_of_week_Wednesday,month_APR,month_AUG,month_JUL,month_JUN,month_MAY,month_OCT,month_SEP,skies_Clear,skies_Cloudy,bobblehead_NO,bobblehead_YES
0,0.716787,-0.466252,2.449490,-0.437595,-0.494413,-0.313112,-0.313112,-0.408248,-0.437595,1.914854,-0.313112,-0.313112,-0.549841,-0.13484,-0.494413,0.494413,-0.494413,0.313112,-0.313112
1,-0.410743,-0.466252,-0.408248,2.285218,-0.494413,-0.313112,-0.313112,-0.408248,-0.437595,1.914854,-0.313112,-0.313112,-0.549841,-0.13484,-0.494413,0.494413,-0.494413,0.313112,-0.313112
2,0.716787,-0.466252,-0.408248,-0.437595,-0.494413,-0.313112,3.193744,-0.408248,-0.437595,1.914854,-0.313112,-0.313112,-0.549841,-0.13484,-0.494413,0.494413,-0.494413,-3.193744,3.193744
3,-1.538273,-0.466252,2.449490,-0.437595,-0.494413,-0.313112,-0.313112,-0.408248,2.285218,-0.522233,-0.313112,-0.313112,-0.549841,-0.13484,-0.494413,-2.022600,2.022600,0.313112,-0.313112
4,1.167799,-0.466252,-0.408248,-0.437595,-0.494413,-0.313112,-0.313112,2.449490,-0.437595,1.914854,-0.313112,-0.313112,-0.549841,-0.13484,-0.494413,0.494413,-0.494413,0.313112,-0.313112


In [38]:
#with test data
scaler_test = preprocessing.StandardScaler().fit(x_test)
scaled_x_test = scaler_test.transform(x_test)
scaled_x_test = pd.DataFrame(scaled_x_test, columns=x_test.columns)

In [39]:
scaled_x_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 19 columns):
temp                     25 non-null float64
day_of_week_Friday       25 non-null float64
day_of_week_Monday       25 non-null float64
day_of_week_Saturday     25 non-null float64
day_of_week_Sunday       25 non-null float64
day_of_week_Thursday     25 non-null float64
day_of_week_Tuesday      25 non-null float64
day_of_week_Wednesday    25 non-null float64
month_APR                25 non-null float64
month_AUG                25 non-null float64
month_JUL                25 non-null float64
month_JUN                25 non-null float64
month_MAY                25 non-null float64
month_OCT                25 non-null float64
month_SEP                25 non-null float64
skies_Clear              25 non-null float64
skies_Cloudy             25 non-null float64
bobblehead_NO            25 non-null float64
bobblehead_YES           25 non-null float64
dtypes: float64(19)
memory usage: 3.8 KB


### 4) Ridge(L2) Regression with leave one out cross validation

In [40]:
model_ridge = RidgeCV(alphas=np.linspace(0.1,1000,100), cv=None, store_cv_values=True)
model_ridge.fit(scaled_x_train, y_train)



RidgeCV(alphas=array([1.000e-01, 1.020e+01, 2.030e+01, 3.040e+01, 4.050e+01, 5.060e+01,
       6.070e+01, 7.080e+01, 8.090e+01, 9.100e+01, 1.011e+02, 1.112e+02,
       1.213e+02, 1.314e+02, 1.415e+02, 1.516e+02, 1.617e+02, 1.718e+02,
       1.819e+02, 1.920e+02, 2.021e+02, 2.122e+02, 2.223e+02, 2.324e+02,
       2.425e+02, 2.526e+02, 2.627e+02, 2.728e+02, 2.829e+02, 2.930e+02,
       3.031e+02, 3.132e+02, 3.233e+0...
       7.273e+02, 7.374e+02, 7.475e+02, 7.576e+02, 7.677e+02, 7.778e+02,
       7.879e+02, 7.980e+02, 8.081e+02, 8.182e+02, 8.283e+02, 8.384e+02,
       8.485e+02, 8.586e+02, 8.687e+02, 8.788e+02, 8.889e+02, 8.990e+02,
       9.091e+02, 9.192e+02, 9.293e+02, 9.394e+02, 9.495e+02, 9.596e+02,
       9.697e+02, 9.798e+02, 9.899e+02, 1.000e+03]),
        cv=None, fit_intercept=True, gcv_mode=None, normalize=False,
        scoring=None, store_cv_values=True)

## Q2 Best L2 regularization coefficient and its corresponding linear coefficients.

In [41]:
print('alpha = ', model_ridge.alpha_) 
print('coef = ', model_ridge.coef_)
l2_predict = model_ridge.predict(scaled_x_test)
print('R2= ',r2_score(y_test,l2_predict))


alpha =  50.6
coef =  [[  515.85380007    67.69322043  -835.67182144   490.25315842
    238.14165655  -341.53918562  1070.28517076  -617.19884398
   -273.56102806   457.54654532  -216.88248689  1102.57100111
   -605.71237164   -75.78927596  -186.41964828   271.19412682
   -271.19412682 -1145.58152754  1145.58152754]]
R2=  0.10450004637534283


### 5) Lasso(L1) Regression with leave one out cross validation

In [42]:
model_lasso = LassoCV(alphas=np.linspace(0.1,1000,100), cv=scaled_x_train.shape[0])
model_lasso.fit(scaled_x_train, y_train)

LassoCV(alphas=array([1.000e-01, 1.020e+01, 2.030e+01, 3.040e+01, 4.050e+01, 5.060e+01,
       6.070e+01, 7.080e+01, 8.090e+01, 9.100e+01, 1.011e+02, 1.112e+02,
       1.213e+02, 1.314e+02, 1.415e+02, 1.516e+02, 1.617e+02, 1.718e+02,
       1.819e+02, 1.920e+02, 2.021e+02, 2.122e+02, 2.223e+02, 2.324e+02,
       2.425e+02, 2.526e+02, 2.627e+02, 2.728e+02, 2.829e+02, 2.930e+02,
       3.031e+02, 3.132e+02, 3.233e+0...
       8.485e+02, 8.586e+02, 8.687e+02, 8.788e+02, 8.889e+02, 8.990e+02,
       9.091e+02, 9.192e+02, 9.293e+02, 9.394e+02, 9.495e+02, 9.596e+02,
       9.697e+02, 9.798e+02, 9.899e+02, 1.000e+03]),
        copy_X=True, cv=56, eps=0.001, fit_intercept=True, max_iter=1000,
        n_alphas=100, n_jobs=None, normalize=False, positive=False,
        precompute='auto', random_state=None, selection='cyclic', tol=0.0001,
        verbose=False)

## Q3 What is the best L1 regularization coefficient? Provide the corresponding linear coefficients

In [43]:
print('alpha = ', model_lasso.alpha_)
print('coef = ', model_lasso.coef_)
print('R2 = ', model_lasso.score(scaled_x_test, y_test))


alpha =  484.9
coef =  [  822.02518773     0.         -1166.96635649   174.1769025
     0.          -480.14527534  1382.2566601   -885.41130729
    -0.           753.52095097     0.          1933.39335016
  -124.7660892     -0.            -0.             0.
    -0.         -2640.59881529     0.        ]
R2 =  0.07724321569205583


## Q4 What are the predictive attributes selected as a result of L1 regularization?

#### The solutions of L1 regularization are sparse and many of its estimated coefficients are zero. So the attributes with non-zero coefficient  are selected. The attributes considered are:

1. temp
2. day_of_week_Monday
3. day_of_week_Saturday
4. day_of_week_Thursday
5. day_of_week_Tuesday
6. day_of_week_Wednesday
6. month_AUG
7. month_JUN
8. month_MAY
9. bobblehead_NO
10. bobblehead_YES

### 6) Linear Regression - no regularization

In [44]:
linear_model = LinearRegression() 
linear_model.fit(scaled_x_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

##  Q5 For the models corresponding to the best L2 and L1 regularization coefficients list the following root-mean-square-error (RMSE): (1) RMSE on the training set, (2) expected prediction RMSE obtained during the cross-validation, (3) RMSE on the testing set. For the linear model without regularization list the following RMSE: (1) RMSE on the training set, and (2) RMSE on the testing set.

In [45]:
model_ridge_predict_train = model_ridge.predict(scaled_x_train)
print('Ridge - RMSE train = ', np.sqrt(mean_squared_error(model_ridge_predict_train,y_train)))

model_ridge_predict_test = model_ridge.predict(scaled_x_test)
print('Ridge - RMSE test = ', np.sqrt(mean_squared_error(model_ridge_predict_test,y_test)))

print("Ridge expected prediction RMSE during cross validation = ", np.min(np.sqrt(model_ridge.cv_values_.mean(axis = 0))))

model_lasso_predict_train = model_lasso.predict(scaled_x_train)
print('Lasso - RMSE train = ', np.sqrt(mean_squared_error(model_lasso_predict_train,y_train)))

model_lasso_predict_test = model_lasso.predict(scaled_x_test)
print('Lasso - RMSE test = ', np.sqrt(mean_squared_error(model_lasso_predict_test,y_test)))

print("Lasso expected prediction RMSE during cross validation = ", np.min(np.sqrt(model_lasso.mse_path_.mean(axis = 1))))


linear_model_predict_train = linear_model.predict(scaled_x_train)
print('Linear Model No regularization - RMSE train = ', np.sqrt(mean_squared_error(linear_model_predict_train,y_train)))

linear_model_predict_test = linear_model.predict(scaled_x_test)
print('Linear Model No regularization - RMSE test = ', np.sqrt(mean_squared_error(linear_model_predict_test,y_test)))




Ridge - RMSE train =  5648.864216385293
Ridge - RMSE test =  7797.67936197997
Ridge expected prediction RMSE during cross validation =  6737.660948839507
Lasso - RMSE train =  5464.5268544047685
Lasso - RMSE test =  7915.460987936537
Lasso expected prediction RMSE during cross validation =  7018.958314999469
Linear Model No regularization - RMSE train =  5254.5387249249425
Linear Model No regularization - RMSE test =  7660.824357772815


## Q6. What observations can you make based on the above RMSE?
#### RMSE of testing is greater than training as the model has been built on the train data and thus can predict well on it.
#### L1 and L2 are tying to overfit as the RMSE value for their train set is much less than the expected value

## Q7. Predict the attendance on a clear Monday in June when the expected temperature is 72 for all three models with and without bobbleheads. Does bobblehead promotion have an impact on the attendance?

In [46]:
predict_new=pd.DataFrame([[72,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0]],columns=x_test.columns)

In [47]:
scaled_no_bobblehead= scaler_test.transform(predict_new)

In [48]:
no_bobblehead_predict = model_ridge.predict(scaled_no_bobblehead)
no_bobblehead_predict

array([[39014.30636444]])

In [49]:
no_bobblehead_predict = model_lasso.predict(scaled_no_bobblehead)
no_bobblehead_predict

array([39078.34364866])

In [50]:
no_bobblehead_predict = linear_model.predict(scaled_no_bobblehead)
no_bobblehead_predict

array([[38583.55528661]])

In [51]:
predict_new=pd.DataFrame([[72,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1]],columns=x_test.columns)
scaled_yes_bobblehead= scaler_test.transform(predict_new)

In [52]:
bobblehead_yes_predict = model_ridge.predict(scaled_yes_bobblehead)
bobblehead_yes_predict

array([[44378.98338796]])

In [53]:
bobblehead_yes_predict = model_lasso.predict(scaled_yes_bobblehead)
bobblehead_yes_predict

array([45261.21210986])

In [54]:
bobblehead_yes_predict = linear_model.predict(scaled_yes_bobblehead)
bobblehead_yes_predict

array([[45167.98210513]])

#### From above we can conclude that Bobblehead  affects the attendance as when bobblehead is there, all three methods have detected attendance to be high as compared to when it is not there.